In [1]:
import os
os.chdir(path='../../')

import pandas as pd
import scanpy as sc
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
import scib

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import STForte.helper as stfhelper
from STForte import STForteModel
from STForte.helper import mclust_R
stfhelper.init_mclust_R()

sc.set_figure_params(dpi=120, transparent=True, dpi_save=400, frameon=False, vector_friendly=False, format="pdf", fontsize=24)
trial_name = "trial-DLPFC/multi_slides/"
plot_dir = f"./{trial_name}/plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
sc.settings.figdir = plot_dir
plt.rcParams['font.sans-serif'] = [
    'Helvetica',
    'Arial',
    'sans-serif',]
palette = px.colors.qualitative.Plotly

import warnings
warnings.filterwarnings('ignore') 

/home/wcx99075523/miniconda3/envs/stforte/lib/python3.9/site-packages/lightning_fabric/plugins/environments/xla.py:18: DeprecationWarning: `ModuleAvailableCache` is a special case of `RequirementCache`. Please use `RequirementCache(module=...)` instead.
  from lightning_fabric.accelerators.tpu import _XLA_AVAILABLE, TPUAccelerator
/home/wcx99075523/miniconda3/envs/stforte/lib/python3.9/site-packages/lightning_fabric/__init__.py:29: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/wcx99075523/miniconda3/envs/stforte/lib/python3.9/site-packages/pytorch_lightning/__init__.py:45: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pytorch_lightning')`.
Im

In [2]:
from bokeh.themes import Theme
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import column, row
from bokeh.plotting import curdoc, figure, show
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, export_svg
import plotly.express as px
palette = px.colors.qualitative.Plotly
output_notebook()

Loading BokehJS ...

In [3]:
adata = sc.read_h5ad(f"./{trial_name}outputs/stforte_multi.h5ad")
# adata = sc.read_h5ad(f"./{trial_name}outputs/stforte_multi_673_676.h5ad")
# adata = sc.read_h5ad(f"./{trial_name}outputs/stforte_multi_669_672.h5ad")
# adata = sc.read_h5ad(f"./{trial_name}outputs/stforte_multi_507_510_.h5ad")
adata

AnnData object with n_obs × n_vars = 14364 × 24155
    obs: 'in_tissue', 'array_row', 'array_col', 'spatialLIBD', 'section_id'
    uns: '151673', '151674', '151675', '151676', 'log1p'
    obsm: 'STForte_ATTR', 'STForte_COMB', 'STForte_TOPO', 'spatial'
    layers: 'log1p'

In [4]:
n_clusters = 7
for emb in ['STForte_ATTR', 'STForte_COMB', 'STForte_TOPO']:
    sc.pp.neighbors(adata, use_rep=f"{emb}")
    adata.obsm[f"UMAP_{emb}"] = sc.tl.umap(adata, copy=True).obsm['X_umap']
    mclust_R(adata, n_clusters, obs_add=f"mclust_{emb}", used_obsm=emb)
    adata.obs[f"mclust_{emb}"] = adata.obs[f"mclust_{emb}"].apply(lambda x: f"C{x}").astype("category")
sc.pp.neighbors(adata)
adata.obsm['UMAP_before'] = sc.tl.umap(adata, copy=True).obsm['X_umap']
adata

         Falling back to preprocessing with `sc.pp.pca` and default params.


AnnData object with n_obs × n_vars = 14364 × 24155
    obs: 'in_tissue', 'array_row', 'array_col', 'spatialLIBD', 'section_id', 'mclust_STForte_ATTR', 'mclust_STForte_COMB', 'mclust_STForte_TOPO'
    uns: '151673', '151674', '151675', '151676', 'log1p', 'neighbors'
    obsm: 'STForte_ATTR', 'STForte_COMB', 'STForte_TOPO', 'spatial', 'UMAP_STForte_ATTR', 'UMAP_STForte_COMB', 'UMAP_STForte_TOPO', 'X_pca', 'UMAP_before'
    layers: 'log1p'
    obsp: 'distances', 'connectivities'

In [5]:
p, source = {}, {}
for emb in ['STForte_ATTR', 'STForte_COMB',  'STForte_TOPO', 'before']:
    p[emb], source[emb] = stfhelper.pl.plot_embeddings(adata, basis=f"UMAP_{emb}", color="section_id", title=emb, return_source=True,
                                                       width=240, height=240, size=1)
    l = stfhelper.pl.layout_centroid_label(source[emb], "section_id", font_size=16)
    p[emb].add_layout(l)
    p[emb].axis.visible = False
    p[emb].grid.visible = False
    p[emb].title.align = "center"
    p[emb].title.text_font = "Arial"
    p[emb].outline_line_color = None
grid = row(*[p[emb] for emb in ['STForte_ATTR', 'STForte_COMB', 'STForte_TOPO','before']])
show(grid)

In [6]:
p, source = {}, {}
for emb in ['STForte_ATTR', 'STForte_COMB',  'STForte_TOPO', 'before']:
    p[emb], source[emb] = stfhelper.pl.plot_embeddings(adata, basis=f"UMAP_{emb}", color="spatialLIBD", title=emb, return_source=True,
                                                       width=240, height=240, size=1)
    l = stfhelper.pl.layout_centroid_label(source[emb], "spatialLIBD", font_size=16)
    p[emb].add_layout(l)
    p[emb].axis.visible = False
    p[emb].grid.visible = False
    p[emb].title.align = "center"
    p[emb].title.text_font = "Arial"
    p[emb].outline_line_color = None
grid = row(*[p[emb] for emb in ['STForte_ATTR', 'STForte_COMB', 'STForte_TOPO','before']])
show(grid)

In [7]:
p, source = {}, {}
for emb in ['STForte_ATTR', 'STForte_COMB', 'STForte_TOPO']:
    p[emb], source[emb] = stfhelper.pl.plot_embeddings(adata, basis=f"UMAP_{emb}", color=f"mclust_{emb}", title=emb, return_source=True,
                                                       width=240, height=240, size=1)
    l = stfhelper.pl.layout_centroid_label(source[emb], f"mclust_{emb}", font_size=16)
    p[emb].add_layout(l)
    p[emb].axis.visible = False
    p[emb].grid.visible = False
    p[emb].title.align = "center"
    p[emb].title.text_font = "Arial"
    p[emb].outline_line_color = None
grid = row(*[p[emb] for emb in ['STForte_ATTR', 'STForte_COMB', 'STForte_TOPO']])
show(grid)

In [8]:
import numpy as np
from bokeh.models import Range1d

p_all = []
coord = adata.obsm["spatial"]
label_sub = adata.obs["mclust_STForte_COMB"]
for ss in adata.obs['section_id'].cat.categories:
    ss = f"{ss}"
    coord_sub = coord[adata.obs["section_id"] == ss]
    stat_sub = label_sub[adata.obs["section_id"] == ss]
    source = ColumnDataSource(dict(x=coord_sub[:, 0], y=coord_sub[:, 1],
                                   label=stat_sub))
    p = figure(title=f"Slice No. {int(ss)}", width=240, height=240, toolbar_location=None, match_aspect=True)
    p.square(
        'x', 'y', source=source,
        fill_color=factor_cmap('label', palette=palette, factors=label_sub.cat.categories),
        angle=0, size=2, line_color=None
        )
    p_all.append(p)
grid = row(*p_all)
show(grid)

In [9]:
p_all = []
coord = adata.obsm["spatial"]
label_sub = adata.obs["mclust_STForte_ATTR"]
for ss in adata.obs['section_id'].cat.categories:
    ss = f"{ss}"
    coord_sub = coord[adata.obs["section_id"] == ss]
    stat_sub = label_sub[adata.obs["section_id"] == ss]
    source = ColumnDataSource(dict(x=coord_sub[:, 0], y=coord_sub[:, 1],
                                   label=stat_sub))
    p = figure(title=f"Slice No. {int(ss)}", width=240, height=240, toolbar_location=None, match_aspect=True)
    p.square(
        'x', 'y', source=source,
        fill_color=factor_cmap('label', palette=palette, factors=label_sub.cat.categories),
        angle=0, size=2, line_color=None
        )
    p_all.append(p)
grid = row(*p_all)
show(grid)

In [10]:
p_all = []
coord = adata.obsm["spatial"]
label_sub = adata.obs["mclust_STForte_TOPO"]
for ss in adata.obs['section_id'].cat.categories:
    ss = f"{ss}"
    coord_sub = coord[adata.obs["section_id"] == ss]
    stat_sub = label_sub[adata.obs["section_id"] == ss]     
    source = ColumnDataSource(dict(x=coord_sub[:, 0], y=coord_sub[:, 1],
                                   label=stat_sub))
    p = figure(title=f"Slice No. {int(ss)}", width=240, height=240, toolbar_location=None, match_aspect=True)
    p.square(
        'x', 'y', source=source,
        fill_color=factor_cmap('label', palette=palette, factors=label_sub.cat.categories),
        angle=0, size=2, line_color=None
        )
    p_all.append(p)
grid = row(*p_all)
show(grid)

In [11]:
print("ARI_COMB:",ARI(adata.obs['mclust_STForte_COMB'].to_numpy(), adata.obs['spatialLIBD'].astype(str)))
print("NMI_COMB:",NMI(adata.obs['mclust_STForte_COMB'].to_numpy(), adata.obs['spatialLIBD'].astype(str)))

ARI_COMB: 0.5982304710242632
NMI_COMB: 0.6951680713705387


In [12]:
print("ARI_ATTR:",ARI(adata.obs['mclust_STForte_ATTR'].to_numpy(), adata.obs['spatialLIBD'].astype(str)))
print("NMI_ATTR:",NMI(adata.obs['mclust_STForte_ATTR'].to_numpy(), adata.obs['spatialLIBD'].astype(str))) 

ARI_ATTR: 0.5742836504552049
NMI_ATTR: 0.663092056886274


In [13]:
print("ARI_TOPO:",ARI(adata.obs['mclust_STForte_TOPO'].to_numpy(), adata.obs['spatialLIBD'].astype(str)))
print("NMI_TOPO:",NMI(adata.obs['mclust_STForte_TOPO'].to_numpy(), adata.obs['spatialLIBD'].astype(str)))

ARI_TOPO: 0.4758784962071061
NMI_TOPO: 0.5902850203000194


In [14]:
print(scib.me.ilisi_graph(adata, batch_key="section_id", type_="embed", use_rep="STForte_COMB",n_cores=8))
print(scib.me.ilisi_graph(adata, batch_key="section_id", type_="embed", use_rep="STForte_ATTR",n_cores=8))
print(scib.me.ilisi_graph(adata, batch_key="section_id", type_="embed", use_rep="STForte_TOPO",n_cores=8))

File has no entries. Doing nothing.
0.654666507922881
File has no entries. Doing nothing.
0.6213158886094212
File has no entries. Doing nothing.
0.8141642187805376


In [15]:
scib.me.ilisi_graph(adata, batch_key="section_id", type_="full", n_cores=8)

File has no entries. Doing nothing.


0.5988869206385448

In [16]:
adata.write(f"./{trial_name}outputs/stforte_multi_673_676.h5ad")
# adata.write(f"./{trial_name}outputs/stforte_multi_669_672.h5ad")
# adata.write(f"./{trial_name}outputs/stforte_multi_507_510.h5ad")